In [1]:
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.


In [2]:
# helper function loss function
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
    
embedding_dim = 1 # dimension of embedding    

In [3]:
X_train =  np.load("ML20_Xtr.npy")
X_val =  np.load("ML20_Xval.npy")
X_test =  np.load("ML20_Xtest.npy")

X_train_noisy = X_train + np.random.normal(loc=0.0, scale=1.0, size=X_train.shape)
X_val_noisy = X_val + np.random.normal(loc=0.0, scale=1.0, size=X_val.shape)
X_test_noisy = X_test + np.random.normal(loc=0.0, scale=1.0, size=X_test.shape)

# binary label

In [4]:
# load labels
y_train_bin =  np.load("ML20_ytrBin.npy")
y_val_bin =  np.load("ML20_yvalBin.npy")
y_test_bin =  np.load("ML20_ytestBin.npy")

In [5]:
# normal autoencoder (Selu)

In [5]:
# joint (multi task learning model) loss

from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='selu')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='selu')(hidden_enc)
hidden_dec = Dense(2, activation='selu')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='selu')(hidden_dec)
predictor = Dense(1, activation='selu')(hidden_dec)



# Model(input, output): map the input to its reconstruction
# definition of autoencoder
joint_model = Model(inputs=x_input, outputs=[reconstructed_input, predictor])


joint_model.summary()

joint_model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

joint_model.fit(
    x = [X_train_noisy], 
    y = [X_train, y_train_bin],
    epochs = 1,
    batch_size = 32,
    validation_data = ([X_val_noisy], [X_val, y_val_bin])
)
joint_model.evaluate([X_test_noisy], [X_test, y_test_bin])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 2)            6           input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 1)            3           dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 2)            4           dense_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (D

[27428.01192001198, 27427.454126370034, 0.5578099751528006]

In [7]:
# 2 single (consecutive) loss

# separate
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)



x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='selu')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='selu')(hidden_enc)
hidden_dec = Dense(2, activation='selu')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='selu')(hidden_dec)


# Model(input, output): map the input to its reconstruction
# definition of autoencoder
autoencoder = Model(x_input, reconstructed_input)


# map the input to its embedding (encoded representation)
encoder = Model(x_input, embedding)


autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss=root_mean_squared_error)

autoencoder.fit(
    X_train_noisy, X_train,
    epochs=5,
    batch_size=32,
    validation_data = (X_val_noisy, X_val)
)

# part 2
predictor = Dense(1, activation='selu')(embedding)


model = Model(x_input, predictor)

model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

model.fit(
    X_train_noisy, y_train_bin,
    epochs=1,
    batch_size=32,
    validation_data = (X_val_noisy, y_val_bin)
)
model.evaluate(X_test_noisy, y_test_bin)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database',)).History will not be written to the database.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 6         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 3         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 4         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 6         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________

nan

In [8]:
# normal autoencoder (sigmoid)

In [9]:
# joint (multi task learning model) loss

from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='sigmoid')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='sigmoid')(hidden_enc)
hidden_dec = Dense(2, activation='sigmoid')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='sigmoid')(hidden_dec)
predictor = Dense(1, activation='sigmoid')(hidden_dec)



# Model(input, output): map the input to its reconstruction
# definition of autoencoder
joint_model = Model(inputs=x_input, outputs=[reconstructed_input, predictor])


joint_model.summary()

joint_model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

joint_model.fit(
    x = [X_train_noisy], 
    y = [X_train, y_train_bin],
    epochs = 5,
    batch_size = 32,
    validation_data = ([X_val_noisy], [X_val, y_val_bin])
)
joint_model.evaluate([X_test_noisy], [X_test, y_test_bin])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 2)            6           input_3[0][0]                    
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 1)            3           dense_11[0][0]                   
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 2)            4           dense_12[0][0]                   
__________________________________________________________________________________________________
dense_14 (

[nan, nan, nan]

In [11]:
# 2 single (consecutive) loss

# separate
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)


x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='sigmoid')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='sigmoid')(hidden_enc)
hidden_dec = Dense(2, activation='sigmoid')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='sigmoid')(hidden_dec)


# Model(input, output): map the input to its reconstruction
# definition of autoencoder
autoencoder = Model(x_input, reconstructed_input)


# map the input to its embedding (encoded representation)
encoder = Model(x_input, embedding)


autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss=root_mean_squared_error)

autoencoder.fit(
    X_train_noisy, X_train,
    epochs=5,
    batch_size=32,
    validation_data = (X_val_noisy, X_val)
)

# part 2
predictor = Dense(1, activation='sigmoid')(embedding)

model = Model(x_input, predictor)

model.summary()

model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

model.fit(
    X_train_noisy, y_train_bin,
    epochs=1,
    batch_size=32,
    validation_data = (X_val_noisy, y_val_bin)
)
model.evaluate(X_test_noisy, y_test_bin)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 6         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 3         
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 4         
_________________________________________________________________
dense_24 (Dense)             (None, 2)                 6         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
Train on 12000157 samples, validate on 4000053 samples
Epoch 1/5
12000157/12000157 [==============================] - 342s 29us/step - loss: 51183

nan

In [ ]:
# normal autoencoder (linear/identity)

In [7]:
# joint (multi task learning model) loss

from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='linear')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='linear')(hidden_enc)
hidden_dec = Dense(2, activation='linear')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='linear')(hidden_dec)
predictor = Dense(1, activation='linear')(hidden_dec)



# Model(input, output): map the input to its reconstruction
# definition of autoencoder
joint_model = Model(inputs=x_input, outputs=[reconstructed_input, predictor])


joint_model.summary()

joint_model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

joint_model.fit(
    x = [X_train_noisy], 
    y = [X_train, y_train_bin],
    epochs = 5,
    batch_size = 32,
    validation_data = ([X_val_noisy], [X_val, y_val_bin])
)
joint_model.evaluate([X_test_noisy], [X_test, y_test_bin])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
dense_11 (Dense)                (None, 2)            6           input_3[0][0]                    
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 1)            3           dense_11[0][0]                   
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 2)            4           dense_12[0][0]                   
__________________________________________________________________________________________________
dense_14 (

[5782.580737405938, 5767.72719324091, 14.853544137106288]

In [16]:
# 2 single (consecutive) loss

# separate
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)


x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='linear')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='linear')(hidden_enc)
hidden_dec = Dense(2, activation='linear')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='linear')(hidden_dec)


# Model(input, output): map the input to its reconstruction
# definition of autoencoder
autoencoder = Model(x_input, reconstructed_input)


# map the input to its embedding (encoded representation)
encoder = Model(x_input, embedding)


autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss=root_mean_squared_error)

autoencoder.fit(
    X_train_noisy, X_train,
    epochs=3,
    batch_size=32,
    validation_data = (X_val_noisy, X_val)
)

# part 2
predictor = Dense(1, activation='linear')(embedding)

model = Model(x_input, predictor)

model.summary()

model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

model.fit(
    X_train_noisy, y_train_bin,
    epochs=1,
    batch_size=32,
    validation_data = (X_val_noisy, y_val_bin)
)
model.evaluate(X_test_noisy, y_test_bin)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 2)                 0         
_________________________________________________________________
dense_51 (Dense)             (None, 2)                 6         
_________________________________________________________________
dense_52 (Dense)             (None, 1)                 3         
_________________________________________________________________
dense_53 (Dense)             (None, 2)                 4         
_________________________________________________________________
dense_54 (Dense)             (None, 2)                 6         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
Train on 12000157 samples, validate on 4000053 samples
Epoch 1/3
12000157/12000157 [==============================] - 400s 33us/step - loss: 5883.

1.035776395571146

# continuous label (-10, 10)

In [8]:
# load labels
y_train_cat =  np.load("ML20_ytrCat.npy")
y_val_cat =  np.load("ML20_yvalCat.npy")
y_test_cat =  np.load("ML20_ytestCat.npy")

In [ ]:
# normal autoencoder (Selu)

In [15]:
# joint (multi task learning model) loss

from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='selu')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='selu')(hidden_enc)
hidden_dec = Dense(2, activation='selu')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='selu')(hidden_dec)
predictor = Dense(1, activation='selu')(hidden_dec)



# Model(input, output): map the input to its reconstruction
# definition of autoencoder
joint_model = Model(inputs=x_input, outputs=[reconstructed_input, predictor])


joint_model.summary()

joint_model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

joint_model.fit(
    x = [X_train_noisy], 
    y = [X_train, y_train_cat],
    epochs = 1,
    batch_size = 32,
    validation_data = ([X_val_noisy], [X_val, y_val_cat])
)
joint_model.evaluate([X_test_noisy], [X_test, y_test_cat])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 2)            0                                            
__________________________________________________________________________________________________
dense_46 (Dense)                (None, 2)            6           input_10[0][0]                   
__________________________________________________________________________________________________
dense_47 (Dense)                (None, 1)            3           dense_46[0][0]                   
__________________________________________________________________________________________________
dense_48 (Dense)                (None, 2)            4           dense_47[0][0]                   
__________________________________________________________________________________________________
dense_49 (

[27429.136521796067, 27427.454247907484, 1.682275000874933]

In [10]:
# 2 single (consecutive) loss

# separate
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)



x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='selu')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='selu')(hidden_enc)
hidden_dec = Dense(2, activation='selu')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='selu')(hidden_dec)


# Model(input, output): map the input to its reconstruction
# definition of autoencoder
autoencoder = Model(x_input, reconstructed_input)


# map the input to its embedding (encoded representation)
encoder = Model(x_input, embedding)


autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss=root_mean_squared_error)

autoencoder.fit(
    X_train_noisy, X_train,
    epochs=5,
    batch_size=32,
    validation_data = (X_val_noisy, X_val)
)

# part 2
predictor = Dense(1, activation='selu')(embedding)

model = Model(x_input, predictor)

model.summary()

model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

model.fit(
    X_train_noisy, y_train_cat,
    epochs=1,
    batch_size=32,
    validation_data = (X_val_noisy, y_val_cat)
)
model.evaluate(X_test_noisy, y_test_cat)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 6         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 3         
_________________________________________________________________
dense_23 (Dense)             (None, 2)                 4         
_________________________________________________________________
dense_24 (Dense)             (None, 2)                 6         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
Train on 12000157 samples, validate on 4000053 samples
Epoch 1/5
12000157/12000157 [==============================] - 473s 39us/step - loss: 32474

1.6818149224208303

In [11]:
# joint (multi task learning model) loss

from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='sigmoid')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='sigmoid')(hidden_enc)
hidden_dec = Dense(2, activation='sigmoid')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='sigmoid')(hidden_dec)
predictor = Dense(1, activation='sigmoid')(hidden_dec)



# Model(input, output): map the input to its reconstruction
# definition of autoencoder
joint_model = Model(inputs=x_input, outputs=[reconstructed_input, predictor])


joint_model.summary()

joint_model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

joint_model.fit(
    x = [X_train_noisy], 
    y = [X_train, y_train_cat],
    epochs = 5,
    batch_size = 32,
    validation_data = ([X_val_noisy], [X_val, y_val_cat])
)
joint_model.evaluate([X_test_noisy], [X_test, y_test_cat])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
dense_26 (Dense)                (None, 2)            6           input_6[0][0]                    
__________________________________________________________________________________________________
dense_27 (Dense)                (None, 1)            3           dense_26[0][0]                   
__________________________________________________________________________________________________
dense_28 (Dense)                (None, 2)            4           dense_27[0][0]                   
__________________________________________________________________________________________________
dense_29 (

[51183.85516363686, 51176.80509230909, 7.050071289855051]

In [12]:
# 2 single (consecutive) loss

# separate
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)


x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='sigmoid')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='sigmoid')(hidden_enc)
hidden_dec = Dense(2, activation='sigmoid')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='sigmoid')(hidden_dec)


# Model(input, output): map the input to its reconstruction
# definition of autoencoder
autoencoder = Model(x_input, reconstructed_input)


# map the input to its embedding (encoded representation)
encoder = Model(x_input, embedding)


autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss=root_mean_squared_error)

autoencoder.fit(
    X_train_noisy, X_train,
    epochs=5,
    batch_size=32,
    validation_data = (X_val_noisy, X_val)
)

# part 2
predictor = Dense(1, activation='sigmoid')(embedding)

model = Model(x_input, predictor)

model.summary()

model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

model.fit(
    X_train_noisy, y_train_cat,
    epochs=1,
    batch_size=32,
    validation_data = (X_val_noisy, y_val_cat)
)
model.evaluate(X_test_noisy, y_test_cat)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_31 (Dense)             (None, 2)                 6         
_________________________________________________________________
dense_32 (Dense)             (None, 1)                 3         
_________________________________________________________________
dense_33 (Dense)             (None, 2)                 4         
_________________________________________________________________
dense_34 (Dense)             (None, 2)                 6         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
Train on 12000157 samples, validate on 4000053 samples
Epoch 1/5
12000157/12000157 [==============================] - 395s 33us/step - loss: 51183

7.050071289855051

In [ ]:
# normal autoencoder (linear/identity)

In [14]:
# joint (multi task learning model) loss

from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)

x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='linear')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='linear')(hidden_enc)
hidden_dec = Dense(2, activation='linear')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='linear')(hidden_dec)
predictor = Dense(1, activation='linear')(hidden_dec)



# Model(input, output): map the input to its reconstruction
# definition of autoencoder
joint_model = Model(inputs=x_input, outputs=[reconstructed_input, predictor])


joint_model.summary()

joint_model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

joint_model.fit(
    x = [X_train_noisy], 
    y = [X_train, y_train_cat],
    epochs = 1,
    batch_size = 32,
    validation_data = ([X_val_noisy], [X_val, y_val_cat])
)


joint_model.evaluate([X_test_noisy], [X_test, y_test_cat])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 2)            0                                            
__________________________________________________________________________________________________
dense_41 (Dense)                (None, 2)            6           input_9[0][0]                    
__________________________________________________________________________________________________
dense_42 (Dense)                (None, 1)            3           dense_41[0][0]                   
__________________________________________________________________________________________________
dense_43 (Dense)                (None, 2)            4           dense_42[0][0]                   
__________________________________________________________________________________________________
dense_44 (

[5837.214999513805, 5819.4102176625165, 17.804782669494866]

In [18]:
# 2 single (consecutive) loss

# separate
from numpy.random import seed
seed(0)
from tensorflow import set_random_seed
set_random_seed(0)


x_dim = X_train.shape[1]
x_input = Input(shape=(x_dim,))


hidden_enc = Dense(2, activation='linear')(x_input)
# embedding is the user-item embedding (encoded representation)
embedding = Dense(embedding_dim, activation='linear')(hidden_enc)
hidden_dec = Dense(2, activation='linear')(embedding)
# reconstructed_input is the (decoded) reconstruction of the input 
# (lossy reconstruction)
reconstructed_input = Dense(x_dim, activation='linear')(hidden_dec)


# Model(input, output): map the input to its reconstruction
# definition of autoencoder
autoencoder = Model(x_input, reconstructed_input)


# map the input to its embedding (encoded representation)
encoder = Model(x_input, embedding)


autoencoder.summary()

autoencoder.compile(optimizer='rmsprop', loss=root_mean_squared_error)

autoencoder.fit(
    X_train_noisy, X_train,
    epochs=4,
    batch_size=32,
    validation_data = (X_val_noisy, X_val)
)

# part 2
predictor = Dense(1, activation='linear')(embedding)

model = Model(x_input, predictor)

model.summary()

model.compile(optimizer='rmsprop', loss=root_mean_squared_error)

model.fit(
    X_train_noisy, y_train_cat,
    epochs=1,
    batch_size=32,
    validation_data = (X_val_noisy, y_val_cat)
)
model.evaluate(X_test_noisy, y_test_cat)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 2)                 0         
_________________________________________________________________
dense_61 (Dense)             (None, 2)                 6         
_________________________________________________________________
dense_62 (Dense)             (None, 1)                 3         
_________________________________________________________________
dense_63 (Dense)             (None, 2)                 4         
_________________________________________________________________
dense_64 (Dense)             (None, 2)                 6         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________
Train on 12000157 samples, validate on 4000053 samples
Epoch 1/4
12000157/12000157 [==============================] - 452s 38us/step - loss: 5881.

1.699366742536696